In [21]:
from torch import nn
import torch

import os
from typing import Iterator
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from datasets import load_from_disk, Dataset as HFDataset # Rename to avoid clash

import torch.nn.functional as F

In [2]:
import torch

print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")

Allocated: 0.0 MB
Reserved: 0.0 MB


In [3]:
import os, sys, json, importlib
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

In [4]:
from models.deepseek import model
importlib.reload(model)

from models.deepseek import train
importlib.reload(train)

from models.deepseek import data
importlib.reload(data)

<module 'models.deepseek.data' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/data.py'>

In [6]:
import importlib
importlib.reload(data)

from torch.utils.data import IterableDataset, DataLoader

In [10]:
os.getcwd()

'/home/ubuntu/projects/llm-models/llm-models/notebook'

In [5]:
tinydataset = data.TinyStoriesDataset(data_dir='../../data')

AttributeError: module 'models.deepseek.data' has no attribute 'TinyStoriesDataset'

In [11]:
dataloader = DataLoader(tinydataset, batch_size=2)

In [12]:
for batch in dataloader:
    print(batch)
    break

['One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.\n<|endoftext|>\nOnce upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves t

In [5]:
model_args = model.ModelArgs(expert_type=model.ExpertType(2),
                             dim=512, inter_dim=1365, moe_inter_dim=256,
                             kv_lora_rank=128, qk_nope_head_dim=32, qk_rope_head_dim=16, 
                             v_head_dim=32, 
                             n_routed_experts=16, n_layers=4)

In [5]:
model_args = model.ModelArgs(expert_type=model.ExpertType(2),
                             dim=128, inter_dim=341, moe_inter_dim=64,
                             kv_lora_rank=32, qk_nope_head_dim=8, qk_rope_head_dim=4, 
                             v_head_dim=8, 
                             n_routed_experts=16, n_layers=4)

In [5]:
model_args = model.ModelArgs(expert_type=model.ExpertType(1),
                             dim=128, inter_dim=341, moe_inter_dim=64,
                             kv_lora_rank=32, qk_nope_head_dim=8, qk_rope_head_dim=4, 
                             v_head_dim=8, 
                             n_routed_experts=16, n_layers=4)

In [6]:
training_args = train.TrainingArgs(
    model_args=model_args,
    batch_size=4,
    gradient_accumulation_steps=8,
    max_steps=100000,
    learning_rate=3e-4,
    checkpoint_dir="./checkpoints",
    data_dir="./data"
)

In [24]:
trainer = train.Trainer(training_args)

Total parameters: 33,000,576
Trainable parameters: 33,000,576
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...
Tokenized dataset loaded. Total chunks: 459757
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/validation'...
Tokenized dataset loaded. Total chunks: 4619


In [7]:
import torch.distributed as dist

In [8]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
dist.init_process_group(backend='nccl', rank=0, world_size=1)

In [23]:
from dataclasses import dataclass, asdict
import torch.nn.functional as F

In [59]:
count = 0
for step, batch in enumerate(trainer.train_loader):
    count += 1
    if count > 4:
        break
    break

In [50]:
y.shape


torch.Size([4, 1024])

In [51]:
y = torch.randint(0, 10000, y.shape)

In [52]:
x, y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [60]:
logits = trainer.model(x, start_pos=0)

loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

loss.backward(retain_graph=False)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [57]:
loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

In [55]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [11]:

def enum_to_value(obj):
    if isinstance(obj, Enum):
        return obj.value
    elif isinstance(obj, dict):
        return {k: enum_to_value(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [enum_to_value(v) for v in obj]
    return obj

In [12]:
for step, batch in enumerate(trainer.train_loader):
    print(batch)
    break

[tensor([[ 1263,    13, 40355,  ...,   465,   835,   290],
        [  284, 13502,  2405,  ...,   635, 11040,    13],
        [  356,   460,   526,  ...,   257,  7812,  7838],
        [   11,   644,   460,  ...,   284,  1441,   262]]), tensor([[ 1263,    13, 40355,  ...,   465,   835,   290],
        [  284, 13502,  2405,  ...,   635, 11040,    13],
        [  356,   460,   526,  ...,   257,  7812,  7838],
        [   11,   644,   460,  ...,   284,  1441,   262]])]


In [13]:
x,y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [14]:
logits.shape

NameError: name 'logits' is not defined

In [25]:
logits1 = trainer.model(x, start_pos=0)

In [26]:
loss1 = F.cross_entropy(
                logits1.view(-1, logits1.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

loss1.backward()

In [27]:
logits2 = trainer.model(x, start_pos=0)

In [28]:
loss2 = F.cross_entropy(
                logits2.view(-1, logits2.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

loss2.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [22]:
loss1 = F.cross_entropy(
                logits1.view(-1, logits1.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

In [29]:
trainer.model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-3): 3 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (gate): Gate()
        (experts): ModuleList(
          (0-15): 16 x Expert(
            (w1): Linear()
            (w2): Linear()
            (w3): Linear()
          )
        )
        (shared_experts): MLP(
          (w1): ColumnParallelLinear()
          (w2): RowParallelLinear()

In [9]:
trainer = train.Trainer(training_args)

Total parameters: 33,000,576
Trainable parameters: 33,000,576
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...


/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:165: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.args.use_amp)


Tokenized dataset loaded. Total chunks: 459757
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/validation'...
Tokenized dataset loaded. Total chunks: 4619


In [9]:
# trainer_model = model.Transformer(trainer.args.model_args)

In [11]:
trainer.model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-3): 3 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (gate): Gate()
        (experts): ModuleList(
          (0-15): 16 x Expert(
            (w1): Linear()
            (w2): Linear()
            (w3): Linear()
          )
        )
        (shared_experts): MLP(
          (w1): ColumnParallelLinear()
          (w2): RowParallelLinear()

In [27]:
torch.topk(scores, 6, dim=-1)[1]

RuntimeError: selected index k out of range

In [10]:
trainer.train()

/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:218: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.args.use_amp, dtype=torch.bfloat16):


loss
loss


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [17]:
for name, param in trainer.model.named_parameters():
    print(param)
    break

AcceleratorError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
id(trainer.optimizer)

273394814022960

In [ ]:
import os
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

# --- Configuration ---
MODEL_NAME = "gpt2"
CONTEXT_LENGTH = 1024
PROCESSED_DATA_DIR = "./processed_tinystories"

def tokenize_and_group_dataset(split: str):
    """Loads a split, tokenizes it, and groups it into fixed-size chunks."""
    
    # 1. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    # 2. Load Raw Dataset
    raw_dataset = load_dataset("roneneldan/TinyStories", split=split)

    # 3. Tokenization
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=False) # No truncation yet
    
    tokenized_datasets = raw_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        num_proc=os.cpu_count(),
    )

    # 4. Grouping and Chunking
    def group_texts(examples):
        # Concatenate all lists of token IDs in the batch
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        
        # Drop the last chunk
        total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH
        
        # Split the concatenated list into chunks of CONTEXT_LENGTH
        result = {
            k: [t[i : i + CONTEXT_LENGTH] for i in range(0, total_length, CONTEXT_LENGTH)]
            for k, t in concatenated_examples.items()
        }
        # For CLM, labels are the input IDs
        result["labels"] = result["input_ids"].copy()
        return result

    lm_dataset = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=os.cpu_count(),
    )
    
    # 5. Save the processed dataset
    output_path = os.path.join(PROCESSED_DATA_DIR, split)
    lm_dataset.save_to_disk(output_path)
    print(f"Processed dataset saved to: {output_path}")

    return lm_dataset

# Run this block once to create the processed data files
if not os.path.exists(PROCESSED_DATA_DIR):
    os.makedirs(PROCESSED_DATA_DIR)
    
    print("\n--- Starting Pre-processing for all splits ---")
    tokenize_and_group_dataset("train")
    tokenize_and_group_dataset("validation")
    # TinyStories also has a 'test' split if needed: tokenize_and_group_dataset("test")
    print("--- Pre-processing Complete ---")

/home/ubuntu/projects/llm-models/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL_NAME = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
split = 'train'


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
# 2. Load Raw Dataset
raw_dataset = load_dataset("roneneldan/TinyStories", split=split)

# 3. Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=False) # No truncation yet


In [6]:
tokenized_datasets = raw_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        num_proc=os.cpu_count(),
    )

In [1]:
from transformers import AutoTokenizer

MODEL_NAME = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)




/home/ubuntu/projects/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
ds = data.tokenize_and_group_dataset('train')

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-6939c3ad-41a5479f633cd8c13bea7ca3;2614ec0e-0f21-41dd-ae9f-1d4b9b16b6fb)

Entry Not Found for url: https://huggingface.co/api/models/openai-community/gpt2/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [10]:
CONTEXT_LENGTH = 1024

In [11]:
# 4. Grouping and Chunking
def group_texts(examples):
    # Concatenate all lists of token IDs in the batch
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Drop the last chunk
    total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH
    
    # Split the concatenated list into chunks of CONTEXT_LENGTH
    result = {
        k: [t[i : i + CONTEXT_LENGTH] for i in range(0, total_length, CONTEXT_LENGTH)]
        for k, t in concatenated_examples.items()
    }
    # For CLM, labels are the input IDs
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=os.cpu_count(),
)

Map (num_proc=4): 100%|██████████████████████████████████████████████████████| 2119719/2119719 [12:34<00:00, 2807.67 examples/s]


In [ ]:
lm_dataset = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=os.cpu_count(),
)

In [8]:
i = 0
for text in raw_dataset:
    print(text)
    batch_data = text
    i += 1
    if i > 3:
        break

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}
{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had man

In [9]:
def group_texts(examples):
    """
    Concatenates all texts and splits them into fixed-size chunks. 
    This is standard practice for CLM training (e.g., training a GPT-2 model).
    """
    # Concatenate all lists of token IDs in the batch
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Drop the last chunk if it's smaller than the context length
    total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH
    
    # Split the concatenated list into chunks of CONTEXT_LENGTH
    result = {
        k: [t[i : i + CONTEXT_LENGTH] for i in range(0, total_length, CONTEXT_LENGTH)]
        for k, t in concatenated_examples.items()
    }
    
    # For Causal Language Modeling (CLM), the 'labels' are the 'input_ids'.
    # The Hugging Face Trainer handles the required shifting for loss calculation.
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
batch_data.keys()

dict_keys(['text'])

In [22]:
class TinyStoriesTokenizedDataset(Dataset):
    """
    A PyTorch Dataset class that loads the *pre-processed*, tokenized, 
    and chunked TinyStories dataset from disk.
    
    It returns a dictionary suitable for a Hugging Face Trainer or custom PyTorch loop.
    """
    def __init__(self, split: str = "train", processed_data_dir: str = "./processed_tinystories"):
        """
        Initializes the dataset by loading the specified split from a local directory.

        Args:
            split (str): The dataset split to load ('train', 'validation').
            processed_data_dir (str): The directory where the pre-processed data is saved.
        """
        file_path = os.path.join(processed_data_dir, split)
        print(f"Loading *tokenized* TinyStories dataset split from: '{file_path}'...")
        
        # Load the pre-processed dataset from disk
        try:
            # We load the processed Arrow Table (Dataset)
            self.dataset = lm_dataset
        except Exception as e:
            raise RuntimeError(
                f"Could not load processed dataset split '{split}'. "
                f"Did you run the pre-processing script first? Error: {e}"
            )

        self.split = split
        print(f"Tokenized dataset loaded. Total chunks: {len(self.dataset)}")

    def __len__(self):
        """
        Returns the total number of fixed-length chunks (examples) in the dataset.
        """
        return len(self.dataset)

    def __getitem__(self, idx: int) -> dict:
        """
        Retrieves the token IDs and labels for the chunk at the given index.

        Args:
            idx (int): The index of the chunk to retrieve.

        Returns:
            dict: A dictionary containing 'input_ids' and 'labels' tensors.
        """
        # Retrieve the example from the Hugging Face Dataset
        example = self.dataset[idx]
        
        # Convert the lists of integers (token IDs) into PyTorch Tensors
        # These are the necessary keys for Causal Language Modeling training
        return {
            'input_ids': torch.tensor(example['input_ids'], dtype=torch.long),
            'labels': torch.tensor(example['labels'], dtype=torch.long)
        }

In [23]:
train_dataset = TinyStoriesTokenizedDataset('train')

Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...
Tokenized dataset loaded. Total chunks: 459751


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler


In [27]:
train_sampler = DistributedSampler(
            train_dataset,
            num_replicas=1,
            rank=0,
            shuffle=True
        )

In [28]:
train_loader = DataLoader(
            train_dataset,
            batch_size=8,
            sampler=train_sampler,
            shuffle=(train_sampler is None),
            num_workers=8,
            pin_memory=True
        )

/home/ubuntu/projects/llm-models/llm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [30]:
for t in train_loader:
    print(t)
    break

{'input_ids': tensor([[20037,  1816,   284,  ...,   640,    11,   612],
        [ 2613,    13,  1119,  ...,   606,   290,   531],
        [  290,   766,   508,  ...,   739,   257, 24484],
        ...,
        [  257,  3704,    13,  ...,   389,   407,  3148],
        [  290,  3835,   290,  ..., 35113,   526,   198],
        [10718,  1306,   284,  ...,    11, 11735,    13]]), 'labels': tensor([[20037,  1816,   284,  ...,   640,    11,   612],
        [ 2613,    13,  1119,  ...,   606,   290,   531],
        [  290,   766,   508,  ...,   739,   257, 24484],
        ...,
        [  257,  3704,    13,  ...,   389,   407,  3148],
        [  290,  3835,   290,  ..., 35113,   526,   198],
        [10718,  1306,   284,  ...,    11, 11735,    13]])}


In [ ]:
b